In [4]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

In [96]:
df5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_3.csv")

In [97]:
df5_matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\4\apoe_4_matches.csv")

In [98]:
unique_ids = df5_matches["PatientID"].unique()
unique_ids = list(unique_ids)
len(unique_ids)

3663

In [15]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")
regex = regex["REGEX"].to_list()

In [99]:
print(df5_matches.columns)

Index(['Unnamed: 0', 'EMPI', 'MRN', 'PatientID', 'PatientEncounterID',
       'ContactDTS', 'EncounterTypeDSC', 'NoteID', 'InpatientNoteTypeDSC',
       'buckets', 'NoteCSNID', 'LineNBR', 'NoteTXT', 'regex_location',
       'regex_sent', 'regex_match', 'match_count'],
      dtype='object')


In [100]:
cols =  ["patient_id", "encounters_with_keywords",'dementia', 'cognition', 'memory', 'cdr', 'mmse', 'moca', 'alzheimer', 'cognitive impairment', 'mci', 'cerebellar', 'neurocognitive', 'lewy', "pick's", 'cortical', 'corticobasal', 'cerebral', 'cerebrovascular', 'amnesia']
summary_stats  = pd.DataFrame(pd.np.empty((len(unique_ids), len(cols))) * pd.np.nan) 
summary_stats.columns = cols
summary_stats["patient_id"] = unique_ids
summary_stats

,patient_id,encounters_with_keywords,dementia,cognition,memory,cdr,mmse,moca,alzheimer,cognitive impairment,mci,cerebellar,neurocognitive,lewy,pick's,cortical,corticobasal,cerebral,cerebrovascular,amnesia
0,Z8126942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Z8127097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Z8134199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Z8139370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Z8141732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658,Z9515280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3659,Z9516683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3660,Z9517072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3661,Z9523898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
for i in tqdm(range(len(unique_ids))):
    keyword_list = df5_matches[df5_matches['PatientID'] == unique_ids[i]]['regex_match'].values
    
    match_tuples = []
    for j in range(len(keyword_list)):
        x = keyword_list[j][2:len(keyword_list[j])-2]
        x = x.split("\", \"")
        match_tuples.append((x[0], 0))

    #print(match_tuples)

    freq = {"dementia":0
        ,"cognition":0
        ,"memory":0
        ,"cdr":0
        ,"mmse":0
        ,"moca":0
        ,"alzheimer":0
        ,'cognitive impairment':0
        ,"mci":0
        ,"cerebellar":0
        ,"neurocognitive":0
        ,"lewy":0
        ,"pick's":0
        ,"cortical":0
        ,"corticobasal":0
        ,"cerebral":0
        ,"cerebrovascular":0
        ,"amnesia":0
    }

    for k, v in match_tuples:
        freq.update({str(k.lower()):int(freq[k.lower()] + 1)})

    #print(freq, i, "\n")

    summary_stats["encounters_with_keywords"][i] = len(df5_matches[df5_matches["PatientID"] == unique_ids[i]])

    for k, v in freq.items():
        summary_stats[str(k.lower())][i] = int(v)
        #print(k, v)

100%|██████████████████████████████████████████████████████████████████████████████| 3663/3663 [03:08<00:00, 19.48it/s]


In [102]:
summary_stats

,patient_id,encounters_with_keywords,dementia,cognition,memory,cdr,mmse,moca,alzheimer,cognitive impairment,mci,cerebellar,neurocognitive,lewy,pick's,cortical,corticobasal,cerebral,cerebrovascular,amnesia
0,Z8126942,21.0,2.0,10.0,1.0,0.0,0.0,0.0,1.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Z8127097,33.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0
2,Z8134199,89.0,0.0,37.0,15.0,0.0,0.0,0.0,0.0,2.0,10.0,10.0,0.0,0.0,0.0,5.0,0.0,6.0,4.0,0.0
3,Z8139370,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Z8141732,48.0,0.0,18.0,6.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658,Z9515280,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0
3659,Z9516683,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3660,Z9517072,62.0,1.0,10.0,8.0,0.0,0.0,5.0,0.0,4.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,13.0,2.0,1.0
3661,Z9523898,4.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
summary_stats.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\4\apoe_4_summary_stats_updated.csv")